In [ ]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut

from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/Users/laurenzheidrich/Downloads/','fused_dataset')
SEQUENCE_NUM = 7


config_filtered = KittiOdometryDatasetConfig(
    cache=True,
    dataset_path=DATASET_PATH,
    sam_folder_name="sam_pred_medium",
    correct_scan_calibration=True,
    filters=FilterList(
        [
            KittiGTMovingObjectFilter(
                os.path.join(
                    DATASET_PATH,
                    "sequences",
                    "%.2d" % SEQUENCE_NUM,
                    "labels",
                )
            ),
            RangeFilter(3, 25),
        ]
    ),
)

dataset = KittiOdometryDataset(config_filtered, SEQUENCE_NUM)

Now we aggregate a large point cloud based on (ind_start, ind_end)

In [ ]:
ind_start = 0
ind_end = 1100
minor_voxel_size = 0.05
major_voxel_size = 0.35

pcd_ground, pcd_nonground, all_poses, T_pcd = aggregate_pointcloud(dataset, ind_start, ind_end, ground_segmentation="patchwork", icp=False)
first_position = T_pcd[:3,3]
'''
pcd_ground = o3d.io.read_point_cloud('test_data/aggregated_7/ground7.pcd')
pcd_nonground = o3d.io.read_point_cloud('test_data/aggregated_7/non_ground7.pcd')
with np.load('test_data/aggregated_7/all_poses.npz') as data:
  all_poses = data['all_poses']
  T_pcd = data['T_pcd']
  first_position = T_pcd[:3,3]
'''

pcd_ground_minor = pcd_ground.voxel_down_sample(voxel_size=minor_voxel_size)
pcd_nonground_minor = pcd_nonground.voxel_down_sample(voxel_size=minor_voxel_size)

num_points = np.asarray(pcd_nonground.points).shape[0]
print("num points: ", num_points, "in non-ground pcd")

num_points_minor = np.asarray(pcd_nonground_minor.points).shape[0]
print("num points: ", num_points_minor, "in non-ground pcd with downsampling of", minor_voxel_size)

Now we subsample the poses based on a voxel_size

In [ ]:
all_positions = []
for p in all_poses:
    all_positions.append(tuple(p[:3,3]))

sampled_indices_local = list(subsample_positions(all_positions, voxel_size=1))
sampled_indices_global = list(subsample_positions(all_positions, voxel_size=1) + ind_start)

poses = np.array(all_poses)[sampled_indices_local]
positions = np.array(all_positions)[sampled_indices_local]

Now we can split the point cloud into chunks based on a tbd chunk_size

In [ ]:
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters

pcd_nonground_chunks, indices, center_positions, center_ids, chunk_bounds = chunks_from_pointcloud(pcd_nonground_minor, T_pcd, positions, first_position, sampled_indices_global, chunk_size, overlap)
pcd_ground_chunks, _, _, _, _ = chunks_from_pointcloud(pcd_ground_minor, T_pcd, positions, first_position, sampled_indices_global, chunk_size, overlap)

pcd_nonground_chunks_major_downsampling = []
pcd_ground_chunks_major_downsampling = []

for nonground, ground in zip(pcd_nonground_chunks, pcd_ground_chunks):
    pcd_nonground_chunks_major_downsampling.append(nonground.voxel_down_sample(voxel_size=major_voxel_size))
    pcd_ground_chunks_major_downsampling.append(ground.voxel_down_sample(voxel_size=major_voxel_size))
    print("Downsampled from", np.asarray(nonground.points).shape, "to", np.asarray(pcd_nonground_chunks_major_downsampling[-1].points).shape, "points (non-ground)")
    print("Downsampled from", np.asarray(ground.points).shape, "to", np.asarray(pcd_ground_chunks_major_downsampling[-1].points).shape, "points (ground)")

patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)

In [ ]:
sequence = 20

print("Start of sequence ", sequence)

first_id = patchwise_indices[sequence][0]
center_id = center_ids[sequence]
center_position = center_positions[sequence]
chunk_indices = indices[sequence]

tarl_indices_global, _ = get_indices_feature_reprojection(sampled_indices_global, first_id, adjacent_frames=(15,15)) 

pcd_chunk = pcd_nonground_chunks[sequence]
pcd_ground_chunk = pcd_ground_chunks[sequence]
chunk_major = pcd_nonground_chunks_major_downsampling[sequence]

points_major = np.asarray(chunk_major.points)
num_points_major = points_major.shape[0]   

print(num_points_major, "points in downsampled chunk (major)")

tarl_features = tarl_features_per_patch(dataset, chunk_major, T_pcd, center_position, tarl_indices_global, chunk_size, search_radius=major_voxel_size/2)
zero_rows = ~np.array(tarl_features).any(1)

spatial_distance = cdist(points_major, points_major)
tarl_distance = cdist(tarl_features, tarl_features)
tarl_distance[zero_rows] = 0
tarl_distance[:,zero_rows] = 0

In [ ]:
proximity_threshold = 1 # meters that points can be apart from each other and still be considered neighbors
alpha = 0.0 # weight of the spatial proximity term 
beta = 0.0 # weight of the label similarity term
gamma = 0.0 # weight of the dinov2 feature similarity term
theta = 0.5 # weight of the tarl feature similarity term

mask = np.where(spatial_distance <= proximity_threshold, 1, 0)
tarl_edge_weights = mask * np.exp(-theta * tarl_distance)

A = tarl_edge_weights
print("Adjacency Matrix built")

In [ ]:
# Remove isolated points
chunk_major, A = remove_isolated_points(chunk_major, A)
print(num_points_major - np.asarray(chunk_major.points).shape[0], "isolated points removed")
num_points_major = np.asarray(chunk_major.points).shape[0]

print("Start of normalized Cuts")
grouped_labels = normalized_cut(A, np.arange(num_points_major), T = 0.03)
num_groups = len(grouped_labels)
print("There are", num_groups, "cut regions")

sorted_groups = sorted(grouped_labels, key=lambda x: len(x))
num_points_top3 = np.sum([len(g) for g in sorted_groups[-3:]])
top3_ratio = num_points_top3 / num_points_major
print("Ratio of points in top 3 groups:", top3_ratio)

random_colors = generate_random_colors(600)

pcd_color = np.zeros((num_points_major, 3))

for i, s in enumerate(grouped_labels):
    for j in s:
        pcd_color[j] = np.array(random_colors[i]) / 255

pcd_chunk.paint_uniform_color([0, 0, 0])
colors = kDTree_1NN_feature_reprojection(np.asarray(pcd_chunk.colors), pcd_chunk, pcd_color, chunk_major)
pcd_chunk.colors = o3d.utility.Vector3dVector(colors)

inliers = get_statistical_inlier_indices(pcd_ground_chunk)
ground_inliers = get_subpcd(pcd_ground_chunk, inliers)
mean_hight = np.mean(np.asarray(ground_inliers.points)[:,2])
cut_hight = get_subpcd(ground_inliers, np.where(np.asarray(ground_inliers.points)[:,2] < (mean_hight + 0.2))[0])
cut_hight.paint_uniform_color([0, 0, 0])

merged_chunk = pcd_chunk + cut_hight

index_file = str(center_id).zfill(6) + '.pcd'
file = os.path.join("test_data", index_file)

o3d.io.write_point_cloud(file, merged_chunk, write_ascii=False, compressed=False, print_progress=False)

print("Pointcloud written to file")

In [ ]:
for sequence in range(len(center_ids)):

    print("Start of sequence ", sequence)

    first_id = patchwise_indices[sequence][0]
    center_id = center_ids[sequence]
    center_position = center_positions[sequence]
    chunk_indices = indices[sequence]

    cam_indices_global, hpr_mask_indices = get_indices_feature_reprojection(sampled_indices_global, first_id, adjacent_frames=(16,13))
    tarl_indices_global, _ = get_indices_feature_reprojection(sampled_indices_global, first_id, adjacent_frames=(10,10)) 
    
    pcd_chunk = pcd_nonground_chunks[sequence]
    pcd_ground_chunk = pcd_ground_chunks[sequence]
    chunk_major = pcd_nonground_chunks_major_downsampling[sequence]

    points_major = np.asarray(chunk_major.points)
    num_points_major = points_major.shape[0]   

    print(num_points_major, "points in downsampled chunk (major)")

    tarl_features = tarl_features_per_patch(dataset, chunk_major, T_pcd, center_position, tarl_indices_global, chunk_size, search_radius=major_voxel_size/2)
    no_tarl_mask = ~np.array(tarl_features).any(1)

    cams = ["cam2", "cam3"]

    #sam_features_minor, chunk_minor = image_based_features_per_patch(dataset, pcd_nonground_minor, chunk_indices, T_pcd, cam_indices_global, cams, cam_id=0, hpr_radius=2000, dino=False, rm_perp=0.0)
    #point2dino
    #dinov2_features_minor = dinov2_mean(point2dino)
    
    #sam_features_major = -1 * np.ones((num_points_major, sam_features_minor.shape[1]))
    #dinov2_features_major = np.zeros((num_points_major, dinov2_features_minor.shape[1])) 

    #sam_features_major = kDTree_1NN_feature_reprojection(sam_features_major, chunk_major, sam_features_minor, chunk_minor)
    #dinov2_features_major = kDTree_1NN_feature_reprojection(dinov2_features_major, chunk_major, dinov2_features_minor, chunk_minor)

    #zero_rows = np.sum(~np.array(sam_features_major).any(1))
    #ratio = zero_rows / num_points_major

    #if ratio > 0.3:
    #    print("The ratio of points without image-based features is", ratio, ". Skipping this chunk.")
    #    continue

    spatial_distance = cdist(points_major, points_major)
    #dinov2_distance = cdist(dinov2_features_major, dinov2_features_major)
    tarl_distance = cdist(tarl_features, tarl_features)
    tarl_distance[no_tarl_mask] = 0
    tarl_distance[:,no_tarl_mask] = 0

    proximity_threshold = 1 # meters that points can be apart from each other and still be considered neighbors
    alpha = 0.0 # weight of the spatial proximity term 
    beta = 4.0 # weight of the label similarity term
    gamma = 0.0 # weight of the dinov2 feature similarity term
    theta = 0.5 # weight of the tarl feature similarity term

    #sam_edge_weights, mask = sam_label_distance(sam_features_major, spatial_distance, proximity_threshold, beta)
    #spatial_edge_weights = mask * np.exp(-alpha * spatial_distance)
    #dinov2_edge_weights = mask * np.exp(-gamma * dinov2_distance)
    mask = np.where(spatial_distance <= proximity_threshold, 1, 0)
    tarl_edge_weights = mask * np.exp(-theta * tarl_distance)

    A = tarl_edge_weights
    print("Adjacency Matrix built")

    # Remove isolated points
    chunk_major, A = remove_isolated_points(chunk_major, A)
    print(num_points_major - np.asarray(chunk_major.points).shape[0], "isolated points removed")
    num_points_major = np.asarray(chunk_major.points).shape[0]

    print("Start of normalized Cuts")
    grouped_labels = normalized_cut(A, np.arange(num_points_major), T = 0.03)
    num_groups = len(grouped_labels)
    print("There are", num_groups, "cut regions")

    sorted_groups = sorted(grouped_labels, key=lambda x: len(x))
    num_points_top3 = np.sum([len(g) for g in sorted_groups[-3:]])
    top3_ratio = num_points_top3 / num_points_major
    print("Ratio of points in top 3 groups:", top3_ratio)

    random_colors = generate_random_colors(600)

    pcd_color = np.zeros((num_points_major, 3))

    for i, s in enumerate(grouped_labels):
        for j in s:
            pcd_color[j] = np.array(random_colors[i]) / 255

    pcd_chunk.paint_uniform_color([0, 0, 0])
    colors = kDTree_1NN_feature_reprojection(np.asarray(pcd_chunk.colors), pcd_chunk, pcd_color, chunk_major)
    pcd_chunk.colors = o3d.utility.Vector3dVector(colors)

    inliers = get_statistical_inlier_indices(pcd_ground_chunk)
    ground_inliers = get_subpcd(pcd_ground_chunk, inliers)
    mean_hight = np.mean(np.asarray(ground_inliers.points)[:,2])
    cut_hight = get_subpcd(ground_inliers, np.where(np.asarray(ground_inliers.points)[:,2] < (mean_hight + 0.2))[0])
    cut_hight.paint_uniform_color([0, 0, 0])

    merged_chunk = pcd_chunk + cut_hight

    index_file = str(center_id).zfill(6) + '.pcd'
    file = os.path.join("test_data", index_file)

    o3d.io.write_point_cloud(file, merged_chunk, write_ascii=False, compressed=False, print_progress=False)

    print("Pointcloud written to file")

Now we can merge the chunks to one large Map!

In [ ]:
point_clouds = []

# List all files in the folder
files = os.listdir("test_data")
files.sort()

# Filter files with a .pcd extension
pcd_files = [file for file in files if file.endswith(".pcd")]
pcd_files = pcd_files[:3] + pcd_files[4:]
print(pcd_files)

# Load each point cloud and append to the list
for pcd_file in pcd_files:
    file_path = os.path.join("test_data", pcd_file)
    point_cloud = o3d.io.read_point_cloud(file_path)
    point_clouds.append(point_cloud)

In [ ]:
merge = merge_chunks_unite_instances(point_clouds)
o3d.io.write_point_cloud("test_data/merge.pcd", merge, write_ascii=False, compressed=False, print_progress=False)